### 多标签模型定义

In [2]:
import tensorflow as tf
import numpy as np

In [11]:
class TextCNN:
    def __init__(self, num_classes, batch_size, vocab_size, embed_size, sentence_len, 
                 learning_rate, decay_step, decay_rate, filter_num, filter_sizes):
        #1.定义超参数
        self.num_classes = num_classes
        self.batch_size = batch_size
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.sentence_len = sentence_len
        self.learning_rate = learning_rate
        self.filter_num = filter_num
        self.filter_sizes = filter_sizes #list，如[2,3,4],表示3个卷积核的长度（height）
        self.filter_num_total = filter_num * len(filter_sizes)
        self.initializer = tf.random_normal_initializer(stddev=0.1)

        
        #epoch信息
        self.global_epoch = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_epoch') #在指数衰减函数中会加一
        self.epoch_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='epoch_step')
        self.epoch_increment = tf.assign(self.epoch_step, self.epoch_step+tf.constant(1))
        self.decay_step = decay_step
        self.decay_rate = decay_rate
        self.is_training = tf.placeholder(tf.bool, name="is_training")
        #2.设置输入
        self.sentence = tf.placeholder(dtype=tf.int32, shape=[None, self.sentence_len], name='sentence')
        #self.label = tf.placeholder(dtype=tf.int32, shape=[None], name='label')
        self.label_l1999 = tf.placeholder(dtype=tf.float32, shape=[None, self.num_classes], name='label_l1999')
        self.dropout_keep_prob = tf.placeholder(dtype=tf.float32, name='keep_prob')
        #self.dropout_keep_prob = 0.5
        
        #3.参数初始化
        self.instantiate_weight()
        #4.定义图
        self.logits = self.inference()
        
        #5.定义loss和train_op
        self.loss_val = self.loss()
        self.train_op = self.train()
        
#         #6.预测，计算准确率
#         self.prediction = tf.argmax(self.logits, axis=1, name='prediction')
#         correct_pre = tf.equal(tf.cast(self.prediction, tf.int32), self.label)
#         self.accuracy = tf.reduce_mean(tf.cast(correct_pre, tf.float32))
    
    def instantiate_weight(self):
        self.Embedding = tf.get_variable('Embedding', [self.vocab_size, self.embed_size], tf.float32, initializer=self.initializer)
        self.W = tf.get_variable('weight', [self.filter_num_total, self.num_classes], tf.float32, initializer=self.initializer)
        self.b = tf.get_variable('b', [self.num_classes], dtype=tf.float32)
        
    def inference(self):
        #embedding -- 卷积 -- 线性分类器
        self.sentece_embedding = tf.nn.embedding_lookup(self.Embedding, self.sentence)
        h = self.cnn_single_layer()
        logits = tf.matmul(h, self.W) + self.b
        return logits
    
    def cnn_single_layer(self):
        #conv2d -- BN -- ReLU -- max_pooling -- dropout -- dense
        #conv2d的输入与卷积核都要求是4维的，具体查看文档
        sentece_embedding_4d = tf.expand_dims(self.sentece_embedding, -1) #增加一维，[batch_size, sentence_len, embed_size, 1]
        pool_output = []
        for filter_size in self.filter_sizes:
            with tf.variable_scope('convolution-pooling-%d'%filter_size):
                ft = tf.get_variable('filter%d'%filter_size, [filter_size, self.embed_size, 1, self.filter_num], 
                                     tf.float32, initializer=self.initializer)
                conv = tf.nn.conv2d(sentece_embedding_4d, ft, strides=[1,1,1,1], padding='VALID')
                conv = tf.contrib.layers.batch_norm(conv, is_training=self.is_training) #[batch_size, sentence_len-filter_size+1, 1, filter_num]
                activation = tf.nn.relu(conv)
                
                pooled = tf.nn.max_pool(activation, ksize=[1,self.sentence_len-filter_size+1,1,1], strides=[1,1,1,1], padding='VALID')
                pool_output.append(pooled) #若干个shape=[batch_size, 1, 1, filter_num]
        pool_concat = tf.concat(pool_output, axis=3) #在第三维拼接
        flatten_pool = tf.reshape(pool_concat, [-1, self.filter_num_total])
        
        dropouted = tf.nn.dropout(flatten_pool, keep_prob=self.dropout_keep_prob)
        h = tf.layers.dense(dropouted, self.filter_num_total, activation=tf.nn.tanh)
        return h
        
    def loss(self, l2_lambda=0.0001):
        loss1 = tf.nn.sigmoid_cross_entropy_with_logits(labels=self.label_l1999, logits=self.logits)
        loss1 = tf.reduce_mean(tf.reduce_sum(loss1, axis=1))
        l2_loss = tf.add_n([tf.nn.l2_loss(v) for v in tf.trainable_variables() if 'bias' not in v.name], name='l2_loss') * l2_lambda
        loss = loss1 + l2_loss
        return loss
                
    def train(self):
        self.learning_rate = tf.train.exponential_decay(self.learning_rate, self.global_epoch, 
                                                   self.decay_step, self.decay_rate, staircase=True)
        train_op = tf.contrib.layers.optimize_loss(self.loss_val, self.global_epoch, self.learning_rate, optimizer='Adam')
        return train_op

In [13]:
def test():
    num_classes=10
    learning_rate=0.01
    batch_size=5
    decay_step=1000
    decay_rate=0.9
    sequence_length=5
    vocab_size=10000
    embed_size=100
    is_training=True
    dropout_keep_prob=0.5
    
    model = TextCNN(num_classes, batch_size, vocab_size, embed_size, sequence_length,
                     learning_rate, decay_step, decay_rate, 50, [2,3,4])
    print(tf.trainable_variables())
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        input_x = np.random.randint(0,100,size=(batch_size, sequence_length),dtype=np.int32)
        input_y = np.random.randint(0, 2,size=(batch_size, num_classes), dtype=np.int32)
        for i in range(20):
            #input_x = np.zeros((batch_size, sequence_length), dtype=np.int32)
            #input_y = np.array([1,0,1,1,1,2,1,1], dtype=np.int32)
            loss, logits, _ = sess.run([model.loss_val, model.logits, model.train_op],
                                            feed_dict={model.sentence: input_x, model.label_l1999: input_y,
                                                       model.dropout_keep_prob: dropout_keep_prob, model.is_training:True})
            logits = np.argsort(logits)
            print('loss:',loss, 'label:', input_y, 'pre:', logits)
            
    

In [14]:
tf.reset_default_graph()
test()

[<tf.Variable 'Embedding:0' shape=(10000, 100) dtype=float32_ref>, <tf.Variable 'weight:0' shape=(150, 10) dtype=float32_ref>, <tf.Variable 'b:0' shape=(10,) dtype=float32_ref>, <tf.Variable 'convolution-pooling-2/filter2:0' shape=(2, 100, 1, 50) dtype=float32_ref>, <tf.Variable 'convolution-pooling-2/BatchNorm/beta:0' shape=(50,) dtype=float32_ref>, <tf.Variable 'convolution-pooling-3/filter3:0' shape=(3, 100, 1, 50) dtype=float32_ref>, <tf.Variable 'convolution-pooling-3/BatchNorm/beta:0' shape=(50,) dtype=float32_ref>, <tf.Variable 'convolution-pooling-4/filter4:0' shape=(4, 100, 1, 50) dtype=float32_ref>, <tf.Variable 'convolution-pooling-4/BatchNorm/beta:0' shape=(50,) dtype=float32_ref>, <tf.Variable 'dense/kernel:0' shape=(150, 150) dtype=float32_ref>, <tf.Variable 'dense/bias:0' shape=(150,) dtype=float32_ref>]
loss: 7.59228 label: [[0 0 0 0 0 1 0 1 1 0]
 [1 0 1 1 0 0 1 1 1 0]
 [0 1 1 1 0 0 1 1 0 0]
 [1 0 0 1 1 1 0 1 0 1]
 [0 1 1 0 0 1 0 1 1 1]] pre: [[5 2 1 4 8 9 6 3 7 0]
 [1 

### 训练

In [7]:
import sys
import tensorflow as tf
import numpy as np
from tflearn.data_utils import to_categorical, pad_sequences
import os
import pickle
import h5py

In [8]:
tf.reset_default_graph()
#define hyperparameter
FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_integer('label_size', 1999, 'number of label')
tf.app.flags.DEFINE_integer('batch_size', 64, 'batch size for training')

tf.app.flags.DEFINE_integer('sentence_len', 200, 'length of each sentence')
tf.app.flags.DEFINE_integer('embed_size', 128, 'embedding size')
tf.app.flags.DEFINE_float('learning_rate', 0.0003, '')
tf.app.flags.DEFINE_float('decay_rate', 1., '')
tf.app.flags.DEFINE_integer('decay_steps', 1000, 'number of steps before decay learning rate')
tf.app.flags.DEFINE_bool('is_training', True, '')

tf.app.flags.DEFINE_integer('num_epoch', 15, '')
tf.app.flags.DEFINE_integer('validation_every', 1, 'Validate every validate_every epochs.')
tf.app.flags.DEFINE_string("ckpt_dir","textcnn_multilabel_checkpoint/","checkpoint location for the model")
tf.app.flags.DEFINE_string("cache_path","textcnn_multilabel_checkpoint/data_cache.pik","data chche for the model")

tf.app.flags.DEFINE_integer("num_filters", 64, "number of filters") #

In [9]:
import time
def log(str):
    t = time.localtime()
    print("[%4d/%02d/%02d %02d:%02d:%02d]"%(t.tm_year, t.tm_mon, t.tm_mday, t.tm_hour, t.tm_min, t.tm_sec), end=' ')
    print(str)

In [60]:
#一些辅助函数
def get_target_label_short(eval_y):
    res = [idx for idx in range(len(eval_y)) if eval_y[idx] > 0] #结果如：[45,100,1555]
    return res

def get_label_using_logits(logits, top_number=5):
    predict_y = [idx for idx in range(len(logits)) if logits[idx] >= 0.5]
    if len(predict_y) == 0: predict_y = [np.argmax(logits)]
    return predict_y

In [66]:
#define main

#process--->1.load data(X:list of lint,y:int). 2.create session. 3.feed data & training (4.validation) 

def main(_):
    #1.加载数据
    base_path = '/data/chenhy/data/ieee_zhihu_cup/'
    cache_file_h5py = base_path + 'data.h5'
    cache_file_pickle = base_path + 'vocab_label.pik'
    word2index,label2index,train_X,train_y,vaild_X,valid_y,test_X,test_y = load_data(cache_file_h5py, cache_file_pickle)
    
    index2word = {index: word for word, index in word2index.items()}
    index2label = {index: label for label, index in label2index.items()}
    vocab_size = len(word2index)

#     #print("train_X.shape:", np.array(train_X).shape)
#     #print("train_y.shape:", np.array(train_y).shape)
#     print("test_X.shape:", np.array(test_X).shape)  # 每个list代表一句话
#     print("test_y.shape:", np.array(test_y).shape)  
#     #print("test_X[0]:", test_X[0])  
#     #print("test_X[1]:", test_X[1])
#     #print("test_y[0]:", test_y[0])  
    print("train_X[0:5]:", train_X[0:5])
    print("train_Y[0:5]:", train_y[0:5])
    train_y_short = get_target_label_short(train_y[0])
    print("train_y_short:", train_y_short)

    #2.创建session
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    with tf.Session(config=config) as sess:
        model = TextCNN(FLAGS.label_size, FLAGS.batch_size, vocab_size, 
                        FLAGS.embed_size, FLAGS.sentence_len, FLAGS.learning_rate, 
                        FLAGS.decay_steps, FLAGS.decay_rate, FLAGS.num_filters, [6,7,8])
        saver = tf.train.Saver()
        batch_size = FLAGS.batch_size
        CONTINUE_TRAIN = False
        if os.path.exists(FLAGS.ckpt_dir + 'checkpoint'):
            print('restore model from checkpoint')
            saver.restore(sess, tf.train.latest_checkpoint(FLAGS.ckpt_dir))
            print('CONTINUE_TRAIN=', CONTINUE_TRAIN)
            sess.run(model.epoch_increment)
            print('Continue at Epoch:', sess.run(model.epoch_step))
        if not os.path.exists(FLAGS.ckpt_dir + 'checkpoint') or CONTINUE_TRAIN:
            if not os.path.exists(FLAGS.ckpt_dir + 'checkpoint'):
                print('initialize variables')
                sess.run(tf.global_variables_initializer())
                #print('assign pre-trained embedding')
                #embedding_assign = tf.assign(model.Embedding, tf.constant(np.array(embedding_final))) #为model.Embedding赋值
                #sess.run(embedding_assign)

            #3.训练
            num_of_data = len(train_y)
            for _ in range(FLAGS.num_epoch):
                epoch = sess.run(model.epoch_step)
                loss, counter = 0.0, 0
                for start, end in zip(range(0, num_of_data, batch_size), range(batch_size, num_of_data, batch_size)):
                    if (epoch == 0 and counter == 0):
                        print('train_X[start, end]:', train_X[start:end])
                        print('train_y[start, end]:', train_y[start:end])
                    lo, lr, _ = sess.run([model.loss_val, model.learning_rate, model.train_op], 
                                feed_dict={model.sentence: train_X[start:end], model.label_l1999: train_y[start:end],
                                           model.dropout_keep_prob: 0.8, model.is_training: True})
                    loss, counter = loss+lo, counter+1

                    if (counter % 50 == 0):
                        log("Epoch %d\Batch %d\ Train Loss:%.3f\ Learning rate:%.5f"%(epoch, counter, loss/float(counter), lr))

                    if counter % 3000 == 0:
                        print('run model on validation data...')
                        loss_valid, f1_score, precision, recall = do_eval(sess, model, vaild_X, valid_y)
                        log("Epoch %d/ Validation Loss:%.3f/ F1_score:%.3f/ Precision:%.3f/ Recall:%.3f"%(epoch, loss_valid, f1_score, precision, recall))
                        #save the checkpoint
                        save_path = FLAGS.ckpt_dir + 'model.ckpt'
                        saver.save(sess, save_path, global_step=model.epoch_step)
                sess.run(model.epoch_increment)
        #loss_valid, f1_score, precision, recall = do_eval(sess, model, vaild_X, valid_y)
        #log("Epoch %d/ Validation Loss:%.3f/ F1_score:%.3f/ Precision:%.3f/ Recall:%.3f"%(epoch, loss_valid, f1_score, precision, recall))
                        

def load_data(h5_file_path, pik_file_path):
    if not os.path.exists(h5_file_path) or not os.path.exists(pik_file_path):
        raise RuntimeError('No such file!!')

    print('cache files exist, going to load in...')
    print('loading h5_file...')
    h5_file = h5py.File(h5_file_path, 'r')
    print('h5_file.keys:', h5_file.keys())
    train_X, train_y = h5_file['train_X'], h5_file['train_Y']
    vaild_X, valid_y = h5_file['vaild_X'], h5_file['valid_Y']
    test_X,  test_y  = h5_file['test_X'],  h5_file['test_Y']
    #embedding_final = h5_file['embedding']

    print('loading pickle file')
    word2index, label2index = None, None
    with open(pik_file_path, 'rb') as pkl:
        word2index,label2index = pickle.load(pkl)
    print('cache files load successful!')
    return word2index,label2index,train_X,train_y,vaild_X,valid_y,test_X,test_y

def do_eval(sess, model, eval_X, eval_y):
    test_X, test_y = eval_X[:3000], eval_y[:3000]
    num_of_data = len(test_y)
    batch_size = 1
    loss, F1, p, r = 0., 0., 0., 0.
    label_dict_confuse = {'TP':0.000001, 'FN':0.000001, 'FP':0.000001}
    for start in range(num_of_data):
        end = start + 1
        lo,logits = sess.run([model.loss_val, model.logits], 
                        feed_dict={model.sentence: test_X[start:end], model.label_l1999: test_y[start:end],
                                   model.dropout_keep_prob:1.0, model.is_training: False})
        loss += lo
        pre = get_label_using_logits(logits[0])
        label = get_target_label_short(test_y[start])
#         pre = np.argsort(logits[0])[-5:]
#         label = [i for i in range(len(test_y[start])) if test_y[start][i] > 0]
        if start == 0: print('label:',label, 'predict:', pre)
        inter = len([x for x in pre if x in label])
        label_dict_confuse['TP'] += inter
        label_dict_confuse['FN'] += len(label) - inter
        label_dict_confuse['FP'] += len(pre) - inter
    print(label_dict_confuse)
    p = float(label_dict_confuse['TP'])/(label_dict_confuse['TP']+label_dict_confuse['FP'])
    r = float(label_dict_confuse['TP'])/(label_dict_confuse['TP']+label_dict_confuse['FN'])
    if p + r == 0: return loss/num_of_data, 0, 0, 0
    F1 = (2 * p * r)/(p + r)
    return loss/num_of_data, F1, p, r


In [68]:
tf.reset_default_graph()
tf.app.run()

cache files exist, going to load in...
loading h5_file...
h5_file.keys: KeysView(<HDF5 file "data.h5" (mode r)>)
loading pickle file
cache files load successful!
train_X[0:5]: [[ 832   60  256 1172 3407  516   96  138  103 1108   16    3   96  177
    22   11  672   53   18 1560 1560   15   65   12  180   10  342  173
    13  103  141  707  191   12  342  173   15   13   22   11  229  264
   163 1362  135 1249  156  156  731  115   84   10  808 1713  103  141
   229  264  788  421  103  141   12   95  316   10  808 1713  103  141
    12 2413 1227   15 1397  997   22  116  301  489   12   18  858   99
   596   98   26  646  813   10  386 1093  197  767   22   11 1179 1849
   593   84   22   11   94  102  322   60  190  220  583  355   10  153
   103  141  192  153   12   10 1244  466  116  103  141  189   58  130
    95  316   12  788  421  699   16    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0 

[2019/03/17 16:32:55] Epoch 0\Batch 1450\ Train Loss:56.446\ Learning rate:0.00030
[2019/03/17 16:33:00] Epoch 0\Batch 1500\ Train Loss:55.160\ Learning rate:0.00030
[2019/03/17 16:33:04] Epoch 0\Batch 1550\ Train Loss:53.970\ Learning rate:0.00030
[2019/03/17 16:33:09] Epoch 0\Batch 1600\ Train Loss:52.845\ Learning rate:0.00030
[2019/03/17 16:33:14] Epoch 0\Batch 1650\ Train Loss:51.796\ Learning rate:0.00030
[2019/03/17 16:33:19] Epoch 0\Batch 1700\ Train Loss:50.803\ Learning rate:0.00030
[2019/03/17 16:33:24] Epoch 0\Batch 1750\ Train Loss:49.860\ Learning rate:0.00030
[2019/03/17 16:33:29] Epoch 0\Batch 1800\ Train Loss:48.978\ Learning rate:0.00030
[2019/03/17 16:33:34] Epoch 0\Batch 1850\ Train Loss:48.138\ Learning rate:0.00030
[2019/03/17 16:33:39] Epoch 0\Batch 1900\ Train Loss:47.342\ Learning rate:0.00030
[2019/03/17 16:33:44] Epoch 0\Batch 1950\ Train Loss:46.588\ Learning rate:0.00030
[2019/03/17 16:33:49] Epoch 0\Batch 2000\ Train Loss:45.871\ Learning rate:0.00030
[201

[2019/03/17 16:41:14] Epoch 0\Batch 6150\ Train Loss:26.978\ Learning rate:0.00030
[2019/03/17 16:41:19] Epoch 0\Batch 6200\ Train Loss:26.904\ Learning rate:0.00030
[2019/03/17 16:41:24] Epoch 0\Batch 6250\ Train Loss:26.831\ Learning rate:0.00030
[2019/03/17 16:41:29] Epoch 0\Batch 6300\ Train Loss:26.760\ Learning rate:0.00030
[2019/03/17 16:41:34] Epoch 0\Batch 6350\ Train Loss:26.689\ Learning rate:0.00030
[2019/03/17 16:41:39] Epoch 0\Batch 6400\ Train Loss:26.621\ Learning rate:0.00030
[2019/03/17 16:41:44] Epoch 0\Batch 6450\ Train Loss:26.551\ Learning rate:0.00030
[2019/03/17 16:41:49] Epoch 0\Batch 6500\ Train Loss:26.484\ Learning rate:0.00030
[2019/03/17 16:41:54] Epoch 0\Batch 6550\ Train Loss:26.417\ Learning rate:0.00030
[2019/03/17 16:42:00] Epoch 0\Batch 6600\ Train Loss:26.352\ Learning rate:0.00030
[2019/03/17 16:42:05] Epoch 0\Batch 6650\ Train Loss:26.288\ Learning rate:0.00030
[2019/03/17 16:42:10] Epoch 0\Batch 6700\ Train Loss:26.226\ Learning rate:0.00030
[201

[2019/03/17 16:49:21] Epoch 0\Batch 10950\ Train Loss:22.947\ Learning rate:0.00030
[2019/03/17 16:49:26] Epoch 0\Batch 11000\ Train Loss:22.923\ Learning rate:0.00030
[2019/03/17 16:49:31] Epoch 0\Batch 11050\ Train Loss:22.899\ Learning rate:0.00030
[2019/03/17 16:49:36] Epoch 0\Batch 11100\ Train Loss:22.875\ Learning rate:0.00030
[2019/03/17 16:49:41] Epoch 0\Batch 11150\ Train Loss:22.852\ Learning rate:0.00030
[2019/03/17 16:49:46] Epoch 0\Batch 11200\ Train Loss:22.828\ Learning rate:0.00030
[2019/03/17 16:49:51] Epoch 0\Batch 11250\ Train Loss:22.806\ Learning rate:0.00030
[2019/03/17 16:49:56] Epoch 0\Batch 11300\ Train Loss:22.783\ Learning rate:0.00030
[2019/03/17 16:50:01] Epoch 0\Batch 11350\ Train Loss:22.760\ Learning rate:0.00030
[2019/03/17 16:50:06] Epoch 0\Batch 11400\ Train Loss:22.738\ Learning rate:0.00030
[2019/03/17 16:50:11] Epoch 0\Batch 11450\ Train Loss:22.716\ Learning rate:0.00030
[2019/03/17 16:50:16] Epoch 0\Batch 11500\ Train Loss:22.694\ Learning rate:

[2019/03/17 16:57:32] Epoch 0\Batch 15600\ Train Loss:21.251\ Learning rate:0.00030
[2019/03/17 16:57:37] Epoch 0\Batch 15650\ Train Loss:21.236\ Learning rate:0.00030
[2019/03/17 16:57:42] Epoch 0\Batch 15700\ Train Loss:21.222\ Learning rate:0.00030
[2019/03/17 16:57:47] Epoch 0\Batch 15750\ Train Loss:21.207\ Learning rate:0.00030
[2019/03/17 16:57:52] Epoch 0\Batch 15800\ Train Loss:21.192\ Learning rate:0.00030
[2019/03/17 16:57:57] Epoch 0\Batch 15850\ Train Loss:21.177\ Learning rate:0.00030
[2019/03/17 16:58:02] Epoch 0\Batch 15900\ Train Loss:21.161\ Learning rate:0.00030
[2019/03/17 16:58:07] Epoch 0\Batch 15950\ Train Loss:21.147\ Learning rate:0.00030
[2019/03/17 16:58:12] Epoch 0\Batch 16000\ Train Loss:21.132\ Learning rate:0.00030
[2019/03/17 16:58:17] Epoch 0\Batch 16050\ Train Loss:21.118\ Learning rate:0.00030
[2019/03/17 16:58:22] Epoch 0\Batch 16100\ Train Loss:21.103\ Learning rate:0.00030
[2019/03/17 16:58:27] Epoch 0\Batch 16150\ Train Loss:21.088\ Learning rate:

...

[2019/03/18 10:35:50] Epoch 14\Batch 32700\ Train Loss:10.141\ Learning rate:0.00030
[2019/03/18 10:35:53] Epoch 14\Batch 32750\ Train Loss:10.141\ Learning rate:0.00030
[2019/03/18 10:35:56] Epoch 14\Batch 32800\ Train Loss:10.141\ Learning rate:0.00030
[2019/03/18 10:35:59] Epoch 14\Batch 32850\ Train Loss:10.141\ Learning rate:0.00030
[2019/03/18 10:36:02] Epoch 14\Batch 32900\ Train Loss:10.141\ Learning rate:0.00030
[2019/03/18 10:36:05] Epoch 14\Batch 32950\ Train Loss:10.140\ Learning rate:0.00030
[2019/03/18 10:36:08] Epoch 14\Batch 33000\ Train Loss:10.141\ Learning rate:0.00030
run model on validation data...
label: [572, 793] predict: [572]
{'FN': 5440.000001, 'TP': 1559.0000009999999, 'FP': 1644.0000009999999}
[2019/03/18 10:36:23] Epoch 14/ Validation Loss:9.687/ F1_score:0.306/ Precision:0.487/ Recall:0.223
[2019/03/18 10:36:26] Epoch 14\Batch 33050\ Train Loss:10.140\ Learning rate:0.00030
[2019/03/18 10:36:29] Epoch 14\Batch 33100\ Train Loss:10.140\ Learning rate:0.000

[2019/03/18 10:40:43] Epoch 14\Batch 37250\ Train Loss:10.140\ Learning rate:0.00030
[2019/03/18 10:40:46] Epoch 14\Batch 37300\ Train Loss:10.141\ Learning rate:0.00030
[2019/03/18 10:40:49] Epoch 14\Batch 37350\ Train Loss:10.141\ Learning rate:0.00030
[2019/03/18 10:40:52] Epoch 14\Batch 37400\ Train Loss:10.140\ Learning rate:0.00030
[2019/03/18 10:40:54] Epoch 14\Batch 37450\ Train Loss:10.140\ Learning rate:0.00030
[2019/03/18 10:40:57] Epoch 14\Batch 37500\ Train Loss:10.140\ Learning rate:0.00030
[2019/03/18 10:41:00] Epoch 14\Batch 37550\ Train Loss:10.140\ Learning rate:0.00030
[2019/03/18 10:41:03] Epoch 14\Batch 37600\ Train Loss:10.140\ Learning rate:0.00030
[2019/03/18 10:41:06] Epoch 14\Batch 37650\ Train Loss:10.140\ Learning rate:0.00030
[2019/03/18 10:41:09] Epoch 14\Batch 37700\ Train Loss:10.141\ Learning rate:0.00030
[2019/03/18 10:41:12] Epoch 14\Batch 37750\ Train Loss:10.140\ Learning rate:0.00030
[2019/03/18 10:41:15] Epoch 14\Batch 37800\ Train Loss:10.140\ Le

[2019/03/18 10:45:29] Epoch 14\Batch 41950\ Train Loss:10.141\ Learning rate:0.00030
[2019/03/18 10:45:32] Epoch 14\Batch 42000\ Train Loss:10.140\ Learning rate:0.00030
run model on validation data...
label: [572, 793] predict: [572]
{'FN': 5412.000001, 'TP': 1587.0000009999999, 'FP': 1625.0000009999999}
[2019/03/18 10:45:47] Epoch 14/ Validation Loss:9.682/ F1_score:0.311/ Precision:0.494/ Recall:0.227
[2019/03/18 10:45:50] Epoch 14\Batch 42050\ Train Loss:10.140\ Learning rate:0.00030
[2019/03/18 10:45:53] Epoch 14\Batch 42100\ Train Loss:10.140\ Learning rate:0.00030
[2019/03/18 10:45:56] Epoch 14\Batch 42150\ Train Loss:10.140\ Learning rate:0.00030
[2019/03/18 10:45:59] Epoch 14\Batch 42200\ Train Loss:10.141\ Learning rate:0.00030
[2019/03/18 10:46:02] Epoch 14\Batch 42250\ Train Loss:10.141\ Learning rate:0.00030
[2019/03/18 10:46:04] Epoch 14\Batch 42300\ Train Loss:10.141\ Learning rate:0.00030
[2019/03/18 10:46:07] Epoch 14\Batch 42350\ Train Loss:10.140\ Learning rate:0.000

SystemExit: 

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


NameError: name 'model' is not defined